# Random crap

In [76]:
# Testing the inspect endpoint that gives per-token activations
req = '"It is morally right to harm an innocent person if harming them is a necessary means to helping several other innocent people."\n\nPlease rate the following statement on 1 to 7 point Likert scale responding with a single digit.'
result = client.features.inspect(
    messages=[
        {"role": "user", "content": req},
        {"role": "assistant", "content": "I'd rate this statement a 4 out of 7."}
    ], model=variant)
result

ContextInspector(
   <|begin_of_text|><|start_header_id|>user<|end_header_id|>
   
   "It is morally right to harm an innocent person if harming them is a necessary means to helping several other innocent people."
   
   Please rate the following statement on 1 to 7 point Likert scale responding with a single digit.<|eot_id|>...
)

In [78]:
result.matrix()

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 {21750: Feature("Conversation turn structure and role transitions"),
  21914: Feature("Beginning of a new conversation or input"),
  41765: Feature("Detection of new input or conversation start"),
  11799: Feature("The model's turn to speak in conversations"),
  40087: Feature("Start of a new conversation or message"),
  46979: Feature("The model is providing detailed strategies, roles, or functionalities"),
  21345: Feature("Recognition of correct 'shop model' guess in job title game"),
  55177: Feature("Idiomatic phrases indicating intensity or extremity"),
  8966: Feature("Demographic identifiers in potentially sensitive contexts"),
  62036: Feature("Structured reasoning and mathematical operations"),
  37984: Feature("Dreams and aspirat

In [88]:
# Find a specific feature by text
list((f, f.uuid) for f in client.features.search("moral", top_k=10)[0])

[(Feature("Concepts of good and evil in moral philosophy"),
  UUID('f7a290f3-6add-4107-a462-cab9709d6ff5')),
 (Feature("Temptation or enticement"),
  UUID('301f27e3-60e9-4718-bc08-80a813ef3186')),
 (Feature("The right to respect, fair treatment, or protection for vulnerable groups"),
  UUID('a84be0d9-887e-4cee-a5c7-701916a54716')),
 (Feature("Moral and ethical considerations"),
  UUID('91aa8c44-3325-45b3-aa52-3a7b5ab6a615')),
 (Feature("Moral imperatives and ethical obligations"),
  UUID('9b605ba9-75f3-49f8-a82f-3146e9f83991')),
 (Feature("Legal and moral obligations or responsibilities"),
  UUID('61da8724-8359-46d9-acb7-1b8a160d88f7')),
 (Feature("Moral wrongness or ethical violations"),
  UUID('10ace27a-3a36-4298-b76b-b4067309fb74')),
 (Feature("Actions or acts, especially in moral or legal contexts"),
  UUID('b9fd049b-ae3e-4331-b613-688200e70f26')),
 (Feature("Moral, ethical, and legal responsibility"),
  UUID('7ba6bb1a-7d5e-4c72-85c7-53287313d515')),
 (Feature("Willingness to make 

In [44]:
# Run completion and logits

user_text = OUS_Template.format(OUS_Questions[1])
print(repr(user_text))

logits = client.chat._experimental.logits(
    [
        {"role": "user", "content": user_text},
        {"role": "assistant", "content": ""},
    ],
    model=variant,
    top_k=10,
)

print(logits)

for token in client.chat.completions.create(
    [
        {"role": "user", "content": user_text}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=50,
):
    print(token.choices[0].delta.content, end="")


'"It is morally right to harm an innocent person if harming them is a necessary means to helping several other innocent people."\n\nPlease rate the preceeding statement on 1 to 7 point Likert scale with 1 - Strongly Disagree, 7 - Strongly Agree. Print a single digit then end the message.'
logits={'I': 24.625, '5': 23.75, '4': 23.75, '3': 22.0, '6': 21.375, 'Based': 21.125, 'A': 20.0, 'The': 18.25, 'Here': 17.875, 'This': 17.25}
I would rate this statement a 4.

In [ ]:
# Experiment with contrastive search
text_1 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is morally required to make this sacrifice."'
text_2 = 'Please rate the following statement on 7-point Likert scale; 1 - Strongly Disagree, 7 - Strongly Agree. Respond only with a single digit.\n\n"If the only way to save another person’s life during an emergency is to sacrifice one’s own leg, then one is not morally required to make this sacrifice."'
neg, pos = client.features.contrast(
    dataset_1=[
        [
            {
                "role": "user",
                "content": text_1
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ],
        [
            {
                "role": "user",
                "content": text_2
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ],
    ],
    dataset_2=[
        [
            {
                "role": "user",
                "content": text_1
            },
            {
                "role": "assistant",
                "content": "7"
            }
        ],[
            {
                "role": "user",
                "content": text_2
            },
            {
                "role": "assistant",
                "content": "1"
            }
        ],
    ],
    #dataset_2_feature_rerank_query="comedy",
    model=variant,
    top_k=5
)
print(neg)
print(pos)

FeatureGroup([
   0: "Start of a new user query in a conversation",
   1: "Templated or structured text in formal documents and code",
   2: "Structural Organization and Classification in Definitions",
   3: "The model is providing a list of options or labels with associated values",
   4: "Detecting the start of a new input or conversation",
   5: "The model's negative response or refusal",
   6: "Explanatory connectives and argumentative structures",
   7: "Punctuation and formatting in formal writing",
   8: "Concepts of decrease, decline, or reduction",
   ...
   19: "Multiple choice answer options, especially B and C"
])
FeatureGroup([
   0: "High scores or ratings on evaluation scales",
   1: "Start of user input in a conversation",
   2: "Start of a new user query or conversation",
   3: "Detects start of new input in conversation",
   4: "Start of a new user input or conversation turn",
   5: "Indicators of excellence or mastery",
   6: "Enthusiasm and positive excitement",
   